## PAIR PROGRAMMING ETL II

### Transformación I - Limpieza
---

In [1]:
import requests
import pandas as pd
import numpy as np
import ast  #permite castear un string que dentro tiene diccionarios, o listas o tuplas
from datetime import datetime, timedelta
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 
pd.options.display.max_columns = None

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El **objetivo** de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

In [2]:
df= pd.read_csv('../files/attacks_limpieza_completa.csv', index_col=0)


In [3]:
df_clima = pd.read_csv('../files/clima_paises.csv', index_col=0)

---

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea

In [4]:
# Creamos un nuevo dataframe filtrando por los cinco países 
df_attacks= df[df['country'].isin(['usa','australia','new zealand','south africa', 'papua new guinea'])]

In [5]:
# Comprobamos que solo están esos países
df_attacks['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [6]:
print(f'Nº filas: {df_attacks.shape[0]}\nNº columnas: {df_attacks.shape[1]}')

Nº filas: 4335
Nº columnas: 21


---

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [7]:
df_clima.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,3,1,15,none,0,12,2,NaN,315,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 40, 'speed': ...",1028.0,0.0
1,6,1,15,none,0,14,1,NaN,50,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 325, 'speed':...",1027.0,0.0


- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [8]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [9]:
# Para separar la lista de diccionarios en varias columnas
x = df_clima['rh_profile'].apply(pd.Series)

In [10]:
x.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 1}","{'layer': '850mb', 'rh': 4}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 6}","{'layer': '700mb', 'rh': 6}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 1}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': 6}"
1,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 2}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 4}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 5}","{'layer': '550mb', 'rh': 6}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 3}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': 3}"


In [11]:
# For loop para sacar el nombre de la columna y los valores de las filas
for i in range(len(x.columns)): 
    
    # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

In [12]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [13]:
# Para separar la lista de diccionarios en varias columnas
y = df_clima['wind_profile'].apply(pd.Series)

In [14]:
# For loop para sacar el nombre de la columna y los valores de las filas
for i in range(len(y.columns)): 
    
    # aplicamos el apply,extraemos los valores de la key "layer" y lo almacenamos en dos variables que convertimos a strings
    nombre = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
    nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con dos variables para "guardar" los valores
    valores = list(y[i].apply(pd.Series)["direction"] )
    valores2= list(y[i].apply(pd.Series)["speed"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)
    df_clima.insert(i,nombre2,valores2)

In [15]:
# Eliminamos las columnas que tienen las listas de diccionarios, información duplicada
df_clima.drop(['rh_profile','wind_profile'], axis=1, inplace=True)

In [16]:
print(f'Nº filas: {df_clima.shape[0]}\nNº columnas: {df_clima.shape[1]}')

Nº filas: 320
Nº columnas: 64


In [17]:
# Hacemos un groupby por los países para sacar la media de la información del clima por país.
df_clima = df_clima.groupby('country').mean()

/tmp/ipykernel_3218/2885472671.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima = df_clima.groupby('country').mean()


In [18]:
df_clima.head()

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction200mb,direction250mb,direction300mb,direction350mb,direction400mb,direction450mb,direction500mb,direction550mb,direction600mb,direction650mb,direction700mb,direction750mb,direction800mb,direction850mb,direction900mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,lifted_index,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,highcloud,midcloud,lowcloud,msl_pressure,snow_depth
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
australia,3.656250,3.515625,3.437500,3.328125,3.375000,3.359375,3.250000,3.375000,3.578125,3.734375,4.000000,4.062500,4.203125,4.500000,4.906250,5.531250,164.609375,150.703125,139.375000,123.750000,120.703125,123.984375,131.796875,141.25000,148.906250,139.687500,141.484375,137.890625,141.640625,136.328125,116.718750,121.562500,13.609375,11.437500,8.484375,6.515625,5.765625,4.796875,3.562500,2.406250,2.562500,3.109375,3.078125,1.859375,1.312500,2.093750,3.640625,6.187500,97.5,3.921875,-3.812500,2.953125,25.937500,10.859375,NaN,117.578125,3.234375,-9999.0,-9999.0,-9999.0,1015.062500,0.0
new zealand,3.734375,3.640625,3.562500,3.703125,3.671875,3.687500,3.734375,3.859375,3.968750,4.250000,4.625000,4.953125,5.281250,5.500000,6.562500,7.531250,287.500000,269.296875,239.687500,240.468750,244.921875,228.671875,221.250000,215.78125,219.921875,215.859375,208.203125,208.984375,210.234375,197.421875,173.828125,173.359375,13.390625,13.140625,8.390625,5.390625,4.046875,2.468750,2.265625,2.078125,2.328125,3.359375,4.546875,5.078125,4.312500,3.937500,1.218750,-2.515625,97.5,6.968750,8.718750,3.312500,15.187500,11.156250,NaN,166.093750,3.390625,-9999.0,-9999.0,-9999.0,1015.875000,0.0
papua new guinea,3.765625,4.296875,4.375000,4.109375,3.312500,2.828125,2.421875,2.640625,3.312500,3.843750,3.921875,4.203125,4.281250,4.296875,4.796875,5.109375,220.859375,189.843750,209.765625,236.484375,247.734375,255.000000,263.281250,243.75000,204.296875,156.171875,78.750000,79.062500,80.625000,81.328125,81.640625,84.531250,13.734375,12.109375,9.421875,5.828125,2.890625,0.765625,-0.234375,-0.156250,-1.593750,-1.703125,0.078125,1.234375,2.546875,2.562500,2.062500,2.609375,97.5,4.843750,-0.578125,2.000000,25.937500,11.390625,NaN,85.703125,3.156250,-9999.0,-9999.0,-9999.0,1010.218750,0.0
south africa,2.453125,2.312500,2.265625,2.265625,2.265625,2.375000,2.531250,2.781250,3.171875,3.421875,4.000000,4.296875,4.562500,4.828125,5.156250,5.593750,208.828125,214.375000,221.093750,226.093750,229.843750,223.671875,222.734375,215.31250,210.625000,207.031250,208.359375,192.109375,183.828125,148.281250,136.484375,138.828125,13.015625,9.734375,7.296875,7.609375,7.859375,8.593750,9.046875,7.218750,4.640625,3.593750,3.421875,3.484375,4.640625,4.734375,6.000000,9.671875,97.5,6.437500,2.375000,0.937500,23.578125,10.281250,NaN,123.203125,2.453125,-9999.0,-9999.0,-9999.0,1019.390625,0.0
usa,2.234375,2.578125,2.906250,3.296875,3.546875,3.859375,4.000000,4.203125,4.500000,4.750000,4.921875,5.093750,5.062500,5.218750,5.406250,5.296875,268.203125,256.562500,250.234375,248.203125,253.593750,254.921875,256.093750,257.65625,259.062500,264.843750,255.859375,269.375000,276.875000,260.546875,246.171875,216.718750,4.703125,4.187500,4.562500,4.609375,4.703125,4.546875,4.000000,3.546875,3.437500,3.953125,5.656250,6.484375,7.125000,7.046875,6.406250,4.859375,97.5,5.171875,11.500000,0.296875,12.218750,5.406250,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.812500,0.0


In [19]:
# Vemos que country figura como índice y la reseteamos para poder unir los dataframes por esa columna
df_clima.reset_index(inplace=True)

In [20]:
# Guardamos el dataframe de clima limpio 
df_clima.to_csv('../files/datos_clima_paises_2.csv')

In [21]:
# Unimos los dos dataframes
df_union= df_attacks.merge(df_clima, how= 'inner', on= 'country')

In [22]:
df_union.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction200mb,direction250mb,direction300mb,direction350mb,direction400mb,direction450mb,direction500mb,direction550mb,direction600mb,direction650mb,direction700mb,direction750mb,direction800mb,direction850mb,direction900mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,lifted_index,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,highcloud,midcloud,lowcloud,msl_pressure,snow_depth
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,2.234375,2.578125,2.90625,3.296875,3.546875,3.859375,4.0,4.203125,4.5,4.75,4.921875,5.09375,5.0625,5.21875,5.40625,5.296875,268.203125,256.5625,250.234375,248.203125,253.59375,254.921875,256.09375,257.65625,259.0625,264.84375,255.859375,269.375,276.875,260.546875,246.171875,216.71875,4.703125,4.1875,4.5625,4.609375,4.703125,4.546875,4.0,3.546875,3.4375,3.953125,5.65625,6.484375,7.125,7.046875,6.40625,4.859375,97.5,5.171875,11.5,0.296875,12.21875,5.40625,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.8125,0.0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,2.234375,2.578125,2.90625,3.296875,3.546875,3.859375,4.0,4.203125,4.5,4.75,4.921875,5.09375,5.0625,5.21875,5.40625,5.296875,268.203125,256.5625,250.234375,248.203125,253.59375,254.921875,256.09375,257.65625,259.0625,264.84375,255.859375,269.375,276.875,260.546875,246.171875,216.71875,4.703125,4.1875,4.5625,4.609375,4.703125,4.546875,4.0,3.546875,3.4375,3.953125,5.65625,6.484375,7.125,7.046875,6.40625,4.859375,97.5,5.171875,11.5,0.296875,12.21875,5.40625,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.8125,0.0
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,2.234375,2.578125,2.90625,3.296875,3.546875,3.859375,4.0,4.203125,4.5,4.75,4.921875,5.09375,5.0625,5.21875,5.40625,5.296875,268.203125,256.5625,250.234375,248.203125,253.59375,254.921875,256.09375,257.65625,259.0625,264.84375,255.859375,269.375,276.875,260.546875,246.171875,216.71875,4.703125,4.1875,4.5625,4.609375,4.703125,4.546875,4.0,3.546875,3.4375,3.953125,5.65625,6.484375,7.125,7.046875,6.40625,4.859375,97.5,5.171875,11.5,0.296875,12.21875,5.40625,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.8125,0.0
3,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,8.730109,2.234375,2.578125,2.90625,3.296875,3.546875,3.859375,4.0,4.203125,4.5,4.75,4.921875,5.09375,5.0625,5.21875,5.40625,5.296875,268.203125,256.5625,250.234375,248.203125,253.59375,254.921875,256.09375,257.65625,259.0625,264.84375,255.859375,269.375,276.875,260.546875,246.171875,216.71875,4.703125,4.1875,4.5625,4.609375,4.703125,4.546875,4.0,3.546875,3.4375,3.953125,5.65625,6.484375,7.125,7.046875,6.40625,4.859375,97.5,5.171875,11.5,0.296875,12.21875,5.40625,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.8125,0.0
4,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,4.581659,2.234375,2.578125,2.90625,3.296875,3.546875,3.859375,4.0,4.203125,4.5,4.75,4.921875,5.09375,5.0625,5.21875,5.40625,5.296875,268.203125,256.5625,250.234375,248.203125,253.59375,254.921875,256.09375,257.65625,259.0625,264.84375,255.859375,269.375,276.875,260.546875,246.171875,216.71875,4.703125,4.1875,4.5625,4.609375,4.703125,4.546875,4.0,3.546875,3.4375,3.953125,5.65625,6.484375,7.125,7.046875,6.40625,4.859375,97.5,5.171875,11.5,0.296875,12.21875,5.40625,NaN,195.234375,2.171875,-9999.0,-9999.0,-9999.0,1020.8125,0.0


In [23]:
df_union.shape

(4335, 83)

---

- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [24]:
df_union.to_csv('../files/datos_clima_attacks.csv')